# Nettoyage de Données et Gestion des NaNs

> La plupart du temps, après avoir importé des données, un certain travail est nécessaire avant de pouvoir les rendre facilement exploitable. Ce travail inclut essentiellement la mise en forme de certaines colonnes et la gestion des valeurs manquantes.
>
> Dans ce notebook, vous apprendrez à nettoyer les données pour obtenir un DataFrame propre et facilement exploitable.

- Importer le package `pandas` sous l'abréviation `pd`.
- Lire le fichier `titanic_train.csv`, en précisant d'utiliser la première colonne comme index.

In [1]:
# Importation du module
import pandas as pd

# Chargement de la base titanic et indexation
titanic = pd.read_csv('titanic_train.csv', sep =',', index_col = 0)

> Certaines variables nécessitent parfois quelques opérations avant d'être exploitables. Nous verrons plus bas comment gérer les valeurs manquantes, mais pour l'instant, intéressons-nous au nettoyage d'une variable.
>
> La méthode `replace()` permet de remplacer et renommer les différentes modalités d'une variable. Par exemple si une variable est stockée avec les modalités `'one'`, `'two'`, alors l'appel suivant permettra de mieux traiter la variable en lui assignant les valeurs numériques 1 et 2.
>
> Exemple : `mon_df['var'] = mon_df['var'].replace(['one','two'],[1,2])`.
>
> Il est parfois nécessaire de modifier le type d'une variable. Par exemple, il arrive qu'une variable de type objet, même renommée, reste de type objet, il faut alors modifier son type si l'on veut qu'elle se comporte de la même façon qu'une variable numérique. Cela est possible grâce à la méthode `astype`.
>
> Exemple : `mon_df['var'] = mon_df['var'].astype(float)`.
>
> Certaines colonnes ont des noms trop longs ou compliqués et peuvent être sources d'erreurs, il est alors judicieux de les renommer, par le biais de la méthode `rename`.
>
> Exemple : `mon_df.rename(columns={'old_name': 'new_name'})`.
>
> Attention. Ces méthodes (`rename()`, `replace()`, `astype()`) ne transforment pas les DataFrame auxquels elles s'appliquent mais créent simplement un nouvel objet. Elles nécessitent une réassignation de l'objet si l'on souhaite garder le nom de l'objet initial. Pour modifier directement un DataFrame, certaines méthodes acceptent l'utilisation de l'argument `inplace=True`. C'est le cas de `rename()` et replace`()`, mais pas de `astype()`.
>
> L'argument `inplace=True`. permet d'éviter de créer une copie de notre DataFrame. Il est donc optimal en mémoire et en temps de calcul, mais il faut être très prudent dans son utilisation puisqu'il modifie directement la base et peut entraîner des pertes.
>
> Dans une copie de `titanic` que nous appellerons `titanic_numeric`, effectuer les opérations suivantes:

- Renomer la variable `Embarked` en `EmbNum`.
- Remplacer les modalités `['S','C','Q']` de `EmbNum` par le triplet `[0,1,2]`, en vous assurant que la variable soit de type `float`.
- Faire apparaître la moyenne obtenue pour cette variable.

In [2]:
titanic_numeric = titanic.rename(columns={'Embarked': 'EmbNum'})
titanic_numeric['EmbNum'] = titanic_numeric['EmbNum'].replace(['S', 'C', 'Q'], [0,1,2])
print('Moyenne pour la variable EmbNum: %0.3f.' %titanic_numeric['EmbNum'].mean())

Moyenne pour la variable EmbNum: 0.362.


- Modifier les valeurs de la variable `Survived` de `titanic`, telle que `0` devienne `'No'` et `1` devienne `'Yes'`.
- Afficher le nombre de survivants et de non survivants, soit le nombre d'occurences pour chaque valeur unique dans la variable `Survived`.

In [3]:
titanic['Survived'].replace([0, 1], ['No', 'Yes'], inplace=True)
titanic['Survived'].value_counts()

No     549
Yes    342
Name: Survived, dtype: int64

> Il est souvent intéressant d'effectuer des opérations, parfois compliquées, sur l'ensemble des colonnes/lignes d'un DataFrame. Pour cela, la structure DataFrame de pandas est équipée d'une méthode très utile, nommée `apply(func, axis)`, qui s'utilise de différentes façons .
>
> Exemple 1 : `mon_df.apply(sum, axis = 1)` affiche la somme des éléments de chaque ligne de `df`.
>
> Exemple 2 : `mon_df.apply(max, axis = 0)` affiche l'élément maximum de chaque colonne de `df`.
>
> Ces exemples fonctionnent, mais peuvent être obtenus directement grâce aux méthodes associées. La fonction `apply()` est souvent beaucoup plus utile lorsqu'elle est associée à une fonction `lambda`.
>
> Il y a deux façons principales de définir une fonction en Python:
>
> 1. La manière classique grâce à l'instruction `def`. Exemple:
```
def increment(x): 
    return x+1
```
>
> 2. L'utilisation de l'instruction `lambda`. Exemple:
```
increment = lambda x: x+1
```
>
> La première est très propre mais l'avantage de la seconde est de pouvoir être définie directement au sein d'une autre expression sans définition préalable. Il suffit, pour définir une fonction au sein d'une méthode ou autre fonction par exemple, d'écrire `lambda x: f(x)` où `f(x)` correspond à nimporte quelle opération transformant `x`.
>
> Exemple 3 : `mon_df.apply(lambda x: max(x) - min(x))` renvoie la différence entre le max et le min pour chaque colonne de `df`.
> 
> Exemple 4 : `titanic['Sex'].apply(lambda x: x[:1])` renvoie la 1ère lettre de chaque élément de la variable `Sex`, soit la même variable avec les modalités `['f','m']`.

- Sachant que la fonction `round(x,i)` renvoie l'élément `x` arrondi à `i` chiffres après la virgule (par défaut 0), afficher les 5 premières valeurs de la variable `Fare` arrondies aux dixièmes.

In [4]:
titanic['Fare'].apply(lambda x: round(x,1)).head()

PassengerId
1     7.2
2    71.3
3     7.9
4    53.1
5     8.1
Name: Fare, dtype: float64

> Une méthode également très souvent utilisée et qui est intégrée dans la structure DataFrame est la méthode `rolling`. Cette méthode permet de faire des moyennes mobiles, des sommes mobiles ou toute opération qui utilise les lignes précédentes d'un DataFrame.
>
> `rolling` retourne une structure agrégeant un nombre de lignes donné et ne renvoie un résultat que si on lui applique une méthode telle que `sum` ou `mean`.
>
> Exemple : `mon_df['var'].rolling(5).sum()` renvoie la somme mobile sur 5 éléments 

- Afficher les moyennes mobiles sur 10 éléments de la variable `Age` de titanic.

In [5]:
titanic.Age.rolling(10).mean()

PassengerId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
887   NaN
888   NaN
889   NaN
890   NaN
891   NaN
Name: Age, Length: 891, dtype: float64

> On remarque alors que les moyennes obtenues contiennent énormément de `NaN` (Not a Number, ou valeurs manquantes). Ceci est dû au fait que sur des fenêtres de 10 éléments, souvent au moins une valeur est manquante, et donc le calcul est impossible. Le problème peut être résolu à l'aide du paramètre `min_periods` afin de choisir un nombre minimum de valeurs non manquantes pour effectuer le calcul, mais beaucoup d'autres problèmes peuvent être causés dans le futur si l'on ne gère pas les NaN d'une meilleure façon.
>
> Il est courant d'avoir des valeurs manquantes dans un jeu de données. Pandas contient quelques fonctions et méthodes très utiles pour gérer ces valeurs manquantes.
>
> La fonction `isnull()` du module pandas permet de détecter si des valeurs manquantes sont présentes. Elle prend en argument un tableau et retourne un nouveau tableau avec les variables booléennes: `True` si la case du tableau originale est vide, `False` sinon. Une méthode du même nom existe pour les DataFrames et Series.
> 
> Il est courant d'ajouter à `isnull()` d'autres méthodes pour plus de clarté.
>
> La méthode `any()` par exemple, retourne `True` si au moins un élément d'un dataframe vaut `True`, et `False` sinon.
>
> Exemple : Pour savoir si au moins un élément est manquant dans le data frame, on utilise `pd.isnull(df).any()` ou `df.isnull().any()`.

- Appliquez la fonction `isnull()` à titanic et lui ajouter la méthode `sum()` pour connaître le nombre exact de valeurs manquantes pour chaque colonne du DataFrame.

In [6]:
titanic.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

> 1. La méthode `fillna()` permet de remplacer les valeurs manquantes d'un data frame par la ou les valeurs de votre choix.
>
>  Exemples :
>  - `df.fillna(0)` remplace toutes les valeurs manquantes par des zéros.
>  - `df.fillna(df.mean())` remplace les valeurs manquantes de chaque colonne par la moyenne sur cette colonne
> 
> 2. La méthode `dropna()` permet de supprimer les lignes ou colonnes contenant des valeurs manquantes. Les paramètres `how` et `axis` permettent de préciser de quelle façon et sur quel axe les éléments seront supprimés. Le paramètre `subset` permet de préciser la liste des colonnes/lignes sur lequelles effectuer la recherche de NaN.
>
>  Exemples :
>  - `df.dropna(how = 'any')` supprime toutes les lignes contenant au moins un NA.
>  - `df.dropna(how = 'all', subset = ['col2','col3','col4'])` supprime les lignes vides sur les 3 colonnes mentionnées.
>  - `df.dropna(axis = 1, how = 'all')` supprime les colonnes complètement vides (par défaut axis=0).
>
> 3. La méthode `drop()` permet de supprimer les lignes ou colonnes d'un tableau. A l'inverse on utilise `append()` pour ajouter une ou plusieurs lignes.
>
>  Exemples :
>  - `df.drop(0)` supprime la première ligne.
>  - `df.drop('Name', axis=1)` supprime la colonne `Name` (par défaut axis=0).
>
> Attention : Ces méthodes (`fillna`, `dropna`, `drop`) ne transforment pas le DataFrame en question mais créent simplement un nouvel objet. Pour modifier directement notre DataFrame, on peut utiliser une réassignation ou l'argument `inplace=True`.
>
> On remarque que la variable `Cabin` contient essentiellement des valeurs manquantes ce qui la rend dificile à utiliser pour une analyse. On peut donc faire le choix de la supprimer.

- Créer a partir de `titanic` un nouveau dataframe `titanic_new` dans lequel la colonne `Cabin` est supprimée.

In [7]:
titanic.drop('Cabin', axis=1, inplace=True)
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,No,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,Yes,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,Yes,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,Yes,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,No,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


> La variable `Age` est plus problématique car la suppression des lignes contenant des valeurs manquantes ou bien même la suppression de la colonne entraîneraient une perte importante d'informations pertinentes.

- Remplacer dans `titanic_new` les valeurs manquantes de la variable `Age` par l'âge moyen arrondi à l'unité.

In [8]:
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].mean()).round(1)
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,No,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,Yes,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,Yes,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,Yes,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,No,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


> La variable `Embarked` ne contient que deux variables manquantes.

- Supprimer les lignes de `titanic_new` qui contiennent des valeurs manquantes pour la variable `Embarked`.
- Vérifier que `titanic_new` ne contient plus de valeurs manquantes

In [9]:
titanic.dropna(how='any', subset=['Embarked'], inplace=True)
titanic.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64